# In-context learning and RAG
This is the second notebook for the LLM section of Comp 255.

1. [System prompts](#system-prompts)
2. [Zero-shot prompting](#zero-shot-prompting)
3. [Few-shot prompting](#few-shot-prompting)
    - [Exercise: Writing JSON](#exercise-writing-json)
4. [Structured outputs](#structured-outputs)
5. [Chatbots](#chatbots)
6. [Retrieval Augmented Generation (RAG)](#retrieval-augmented-generation-rag)
    - [Vector stores](#vector-stores)
    - [Exercise: Your turn!](#exercise-your-turn)

In [ ]:
from llamabot import SimpleBot, StructuredBot, ChatBot
import json
from pydantic import BaseModel

sft_model = "qwen2.5:1.5b"

### System prompts
Depending on the model, the "system prompt" section is handled a little differently from the instruction itself.  

You can see the "system" tag in Ollama's [template for Llama3](https://ollama.com/library/llama3/blobs/8ab4849b038c).  This is where the prompts we put below will be inserted.

In [ ]:
surfer = SimpleBot(
    system_prompt='Respond like a surfer dude',
    model_name=f"ollama_chat/{sft_model}",
)

pirate = SimpleBot(
    system_prompt='Respond like a pirate',
    model_name=f"ollama_chat/{sft_model}",
)

print(surfer.system_prompt)
print(pirate.system_prompt)

In [ ]:
response = surfer('How are you today?')
print('\n')
response = pirate('How are you today?')

## Zero-shot prompting
"Zero-shot" learning refers to a model's ability to provide a correct output to a question it wasn't directly trained to answer.  We already sort of have an example of that.

In [ ]:
simple_bot = SimpleBot(
    system_prompt='You are a helpful bot',
  model_name=f"ollama_chat/{sft_model}",
)

response = simple_bot('What is the capital of France?')

But what if we want it to JUST output the name?

In [ ]:
prompt = """Answer the following question, provide no other information:
What is the capital of France?"""
response = simple_bot(prompt)

## Few-shot prompting
The above is fine, but maybe we want our output in a particular format.  We'd be best served by giving the model examples.  This is the "few" in "few-shot" - we're giving the model examples rather than just instructions.

In [ ]:
prompt = """Answer the following question in the following format:

Question: What is the capital of Germany?
Answer: Berlin

Question: What is the capital of France?
Answer: """
response = simple_bot(prompt)

### Exercise: Writing JSON
There may be the case that you want the LLM to output something with some defined structure so you can apply functions neatly on top of it.  One such format is JSON.  

Say you want an output for country information in the following format:

`{"country": str, "capital": str}`

Ask your bot to do this for you! Try in zero shot formulation and few-shot formulation.

## Structured Output
As you can see, sometimes these models are iffy about producing valid, parseable JSON.  Even ChatGPT suffers from this, sometimes outputting some preamble that wouldn't, without processing, be parseable.  That's why it makes sense to think about "structured output", which can be implemented in different ways. In Ollama, the supplied structure is converted into a "grammar" which defines which tokens are valid and which are not.  Based on that, invalid token predictions are ignored and only the (hopefully) valid response is returned.

In [ ]:
class Country(BaseModel):
  # source: https://ollama.com/blog/structured-outputs
  name: str
  capital: str
  languages: list[str]

struct_completer = StructuredBot(
    system_prompt='You are a helpful bot',
    model_name=f"ollama_chat/{sft_model}",
    pydantic_model=Country,
)

response = struct_completer('Tell me about France')

json.loads(response.model_dump_json())

## Chatbots
You'll notice that everything we've done so far is a single request, single response.  There is no "conversation" and that's because the model has no context; it doesn't remember previous inputs or responses.

In [ ]:
pirate = SimpleBot(
    system_prompt='You are a pirate',
    model_name=f"ollama_chat/{sft_model}",
)


In [ ]:
response = pirate('How are you today?')

In [ ]:
response = pirate('What did you just say?')

By using `Llamabot.ChatBot`, we provide the model with context (through the `messages` attribute).  This is inserted into the input to the model and it generates a response that is context-sensitive.

In [ ]:
pirate_chat = ChatBot(
  "You are a pirate",
  session_name="pirate_chat",  
  model_name=f"ollama_chat/{sft_model}",
)

In [ ]:
print(pirate_chat.messages)

In [ ]:
response = pirate_chat('How are you today?')

In [ ]:
print(pirate_chat.messages)

In [ ]:
response = pirate_chat('What did you say?')

In [ ]:
pirate_chat.messages

## Retrieval Augmented Generation (RAG)
You've likely heard some buzz about this concept.  There's a lot of complex ways to implement this, but the basic version is essentially just providing the model context based on the product of a "retrieval" workflow.

Let's use the above as an example.  We're relying on the model's internal knowledge to give us the correct information. This doesn't always work.

In [ ]:
response = struct_completer('Tell me about Papua New Guinea')


If we look up on [Wikipedia](https://en.wikipedia.org/wiki/Languages_of_Papua_New_Guinea), we get a different answer:

"Languages with statutory recognition are Tok Pisin, English, Hiri Motu, and Papua New Guinean Sign Language..." 

### Exercise: Make my model smarter
Think about how we might be able to ensure the model produces the right answer here.  Experiment and see if you can get it to generate the right answer.

In [ ]:
struct_completer("""
Here's some useful context: Languages with statutory recognition are Tok Pisin, English, Hiri Motu, and Papua New Guinean Sign Language
                 
Tell me about Papua New Guinea""")

Guess what, you just did RAG! But I'm guessing you probably don't want to always be the "R" part of the workflow.  In that case, we need to set up automated retrieval and to that we need to set up a document store!

### Vector stores
The first part of RAG is "retrieval".  To do that we essentially need to create a mechanism for the model to retrieve relevant information.  One approach is to create a set of "embeddings" for our documents that can be compared against the embedding of an input prompt.

First let's create some documents.  Let's say one contains information about Papua New Guinea, another contains information about France.

In [ ]:
doc1 = "Languages with statutory recognition in Papua New Guinea \
are Tok Pisin, English, Hiri Motu, and \
Papua New Guinean Sign Language"

doc2 = "The only language with statutory recognition in France \
is French"

# write these to a temporary file
with open("doc1.txt", "w") as f:
    f.write(doc1)

with open("doc2.txt", "w") as f:
    f.write(doc2)

We'll be using an implementation from Llamabot, which uses [LanceDB](https://lancedb.com/) on the backend.  LanceDB has a nice implementation for creating vector stores.  Let's see how that looks:

In [ ]:
import lancedb
from lancedb.pydantic import LanceModel, Vector
from lancedb.embeddings import get_registry

# create a database
db = lancedb.connect("/tmp/db")
# initialize a default sentence-transformers model (paraphrase-MiniLM-L6-v2)
model = get_registry().get("sentence-transformers").create()

# specify a schema (just text + vector)
class Words(LanceModel):
    text: str = model.SourceField()
    vector: Vector(model.ndims()) = model.VectorField()


try:
    table = db.create_table("words", schema=Words)
except ValueError:
    table = db.open_table("words")

# add in some entries
table.add(
    [
        {"text": "hello world"},
        {"text": "goodbye world"}
    ]
)


In [ ]:
# look at the entries
table.head()

In [ ]:
query = "greetings"
search_query = table.search(query)
search_query._query[:10]

In [ ]:
# get a single (most similar) result, translate it into the pydantic model
search_query.limit(1).to_pydantic(Words)[0].text

In [ ]:
query = "farewell"
result = table.search(query).limit(1).to_pydantic(Words)[0]
print(result.text)

In [ ]:
query = "random word"
result = table.search(query).limit(1).to_pydantic(Words)[0]
print(result.text)

Llamabot provides a class called `QueryBot` which implements everything above for you and allows you to just query the vector database.

In [ ]:
from llamabot import QueryBot
from pathlib import Path

system_message = "You are a helpful assistant that can answer questions \
    based on the provided documents."
doc_paths = [Path("doc1.txt"), Path("doc2.txt")]

query_completer = QueryBot(
    system_prompt=system_message,
    model_name=f"ollama_chat/{sft_model}",
    collection_name="documents",
    document_paths=doc_paths
)
print(query_completer.system_prompt.content)

In [ ]:
q = "Tell me about Papua New Guinea"
# what does it retrieve (default n of results is 20)
print(query_completer.docstore.retrieve(q))
response = query_completer('Tell me about Papua New Guinea, very brief')

In [ ]:
response = query_completer('Tell me about France, very brief')

In [ ]:
q = 'Tell me about Germany'
query_completer.docstore.retrieve(q, 2)

In [ ]:
response = query_completer('Tell me about Germany, very brief')

### Exercise: Your turn!
Now that you have a sense of how to build out a simple database and the `QueryBot`, try this yourself.  You can either continue using this setup and use another complex country (e.g. Bolivia), or you can build your own document set and query!